In [1]:
# theme = 'plotly_dark'
theme = 'seaborn'
#theme = 'plotly'

In [2]:
import pandas as pd
import json
import torch



import plotly.express as px
# import plotly.graph_objects as go
# import plotly.subplots as sp
import os

In [3]:
PROJECT_DIR = '/home/kuba/Documents/Data/Raw/Listerine/3_final'  
watch_num = '03'
recording = '2023-07-18_07_21_53'

HERTZ = 100
ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING = {
    'water':0,
    'listerine':1,
}

In [4]:
#takes in label a to tensor for ML
def json_to_tensor(labels_x, acc_len_x, acc_x):
    y_new = torch.zeros(acc_len_x)-1

    bouts = []
    for hand in labels_x:
        for action in labels_x[hand]:
            for bout in labels_x[hand][action]:
                y_new[(acc_x.timestamp > bout['start']) & (acc_x.timestamp < bout['end'])] = (ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING[action] * 20 + 15)
    return y_new



In [5]:
def viz_labels(df_acc, y, title):
    y_df = pd.DataFrame(y,columns=['labels'])
    df = pd.concat([df_acc, y_df], axis=1)
    df = df[::5]
    fig = px.line(df, x='timestamp', y=['x','y','z','labels'], title=title)
    fig.update_layout(template=theme)
    fig.show(renderer='browser')

In [6]:
WATCH_DIR = f'{PROJECT_DIR}/{watch_num}'

if recording == '*':
    for recording in sorted(os.listdir(WATCH_DIR)):
        print(recording)
        if recording == '.DS_Store':
            continue
        recording_dir = f'{WATCH_DIR}/{recording}'
        acc = pd.read_csv(f'{recording_dir}/acceleration.csv',skiprows=1)
        with open(f'{recording_dir}/labels.json','r') as f:
            labels = json.load(f)  
        acc_len = len(acc)
        acc['timestamp'] = ((acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9)
        y = json_to_tensor(labels, acc_len, acc)
        viz_labels(acc, y, recording_dir)
else:
    recording_dir = f'{WATCH_DIR}/{recording}'
    acc = pd.read_csv(f'{recording_dir}/acceleration.csv',skiprows=1)
    with open(f'{recording_dir}/labels.json','r') as f:
        labels = json.load(f)  
    acc_len = len(acc)
    acc['timestamp'] = ((acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9)
    y = json_to_tensor(labels, acc_len, acc)
    viz_labels(acc, y, recording_dir)



